In [208]:
import heapq
import numpy as np
import pandas as pd
import math
from numba import jit

In [209]:
class MinimumHeap:
  """
  A class representing a minimum heap.

  Attributes:
    heap (list): The list representing the minimum heap.

  Methods:
    push(value): Pushes a value into the minimum heap.
    pop(): Removes and returns the minimum value from the heap.
    peek(): Returns the minimum value from the heap without removing it.
    size(): Returns the number of elements in the heap.
  """
  def __init__(self):
    self.heap = []

  def push(self, value):
    heapq.heappush(self.heap, value)

  def pop(self):
    return heapq.heappop(self.heap)

  def peek(self):
    return self.heap[0] if self.heap else None

  def size(self):
    return len(self.heap)

In [210]:
class Graph:
  def __init__(self, numVertices):
    self.numVertices = numVertices
    self.adjM = np.zeros((numVertices, numVertices), dtype=int)
    
  def pushEdge(self, source, destination, weight=None):
    self.adjM[source, destination] = weight
    self.adjM[destination, source] = weight

  def getNeighbors(self, vertex):
    row = self.adjM[vertex]
    indices = np.nonzero(row)
    weights = row[indices]
    return list(zip(indices[0], weights))

  def getEdges(self):
    """
    Returns a list of all edges in the graph.

    Returns:
      A list of tuples representing the edges in the graph.
    """
    indices = np.nonzero(self.adjM)
    edges = list(zip(indices[0], indices[1], self.adjM[indices]))
    return edges

  def __str__(self):
    return str(self.adjM)

In [211]:
def MST_Prim(graph):
  """
  Finds the minimum spanning tree of a graph using Prim's algorithm.

  Args:
    graph: The graph to find the minimum spanning tree of.

  Returns:
    A new graph instance representing the minimum spanning tree of the input
  """
  mst = Graph(graph.numVertices)
  visited = np.zeros(graph.numVertices, dtype=bool)
  heap = MinimumHeap()
  heap.push((0, 0, None))

  while heap.size() > 0:
    weight, vertex, parent = heap.pop()
    if visited[vertex]:
      continue
    visited[vertex] = True
    if parent is not None:
      mst.pushEdge(parent, vertex, weight)
    for neighbor, weight in graph.getNeighbors(vertex):
      if not visited[neighbor]:
        heap.push((weight, neighbor, vertex))
  return mst

In [212]:
def TreePreorderWalk(graph):
  """
  Performs a preorder walk of a tree.

  Args:
    graph: The graph to perform the preorder walk on.

  Returns:
    A list of vertices in the order they were visited.
  """
  visited = np.zeros(graph.numVertices, dtype=bool)
  stack = [0]
  walk = []

  while len(stack) > 0:
    vertex = stack.pop()
    if visited[vertex]:
      continue
    visited[vertex] = True
    walk.append(vertex)
    for neighbor, _ in graph.getNeighbors(vertex):
      if not visited[neighbor]:
        stack.append(neighbor)
  return walk

In [213]:
def approxTSP(graph):
  """
  Computes an approximate solution to the traveling salesman problem using
  minimum spanning trees.

  Args:
    graph: The graph to compute the approximate solution for.

  Returns:
    A list of vertices in the order they are visited.
  """
  mst = MST_Prim(graph)
  walk = TreePreorderWalk(mst)
  walk.append(0)
  return walk

In [214]:
def perfectMatching(graph, consideredVertices=None):
  """
  Computes a minimum weight perfect matching for a graph considering only the vertices in the consideredVertices list.

  Args:
    graph: The graph to compute the minimum weight perfect matching for.
    consideredVertices: A list of vertices to consider for the perfect matching.

  Returns:
    A list of edges representing the minimum weight perfect matching.
  """
  if consideredVertices is None:
    consideredVertices = range(graph.numVertices)
  edges = graph.getEdges()
  edges = [edge for edge in edges if edge[0] in consideredVertices and edge[1] in consideredVertices]
  edges.sort(key=lambda x: x[2])
  matching = []
  visited = np.zeros(graph.numVertices, dtype=bool)
  for edge in edges:
    if edge[0] in consideredVertices and edge[1] in consideredVertices and not visited[edge[0]] and not visited[edge[1]]:
      matching.append(edge)
      visited[edge[0]] = True
      visited[edge[1]] = True
  return matching

In [215]:
def eulerianTour(graph):
  """
  Computes an Eulerian tour for a graph.

  Args:
    graph: The graph to compute the Eulerian tour for.

  Returns:
    A list of vertices in the order they are visited.
  """
  start_vertex = 0  # Choose a starting vertex
  visited = np.zeros(graph.numVertices, dtype=bool)
  tour = []
  stack = np.array([start_vertex])

  while stack.size > 0:
    vertex = stack[-1]
    if not visited[vertex]:
      visited[vertex] = True
      tour.append(vertex)
    neighbors = graph.getNeighbors(vertex)
    unvisited_neighbors = np.array([neighbor for neighbor, _ in neighbors if not visited[neighbor]])
    if unvisited_neighbors.size > 0:
      next_vertex = unvisited_neighbors[0]
      stack = np.append(stack, next_vertex)
    else:
      stack = stack[:-1]

  return tour

In [216]:
def christofidesAlgorithm(graph):
  """
  Computes an approximate solution to the traveling salesman problem using
  Christofides algorithm.

  Args:
    graph: The graph to compute the approximate solution for.

  Returns:
    A list of vertices in the order they are visited.
  """
  def eliminateDuplicated(tour):
    for vertex in tour:
      if tour.count(vertex) > 1:
        tour.remove(vertex)

  mst = MST_Prim(graph)
  oddDegreeVertices = []
  for vertex in range(mst.numVertices):
    if len(mst.getNeighbors(vertex)) % 2 == 1:
      oddDegreeVertices.append(vertex)
  matching = perfectMatching(graph, oddDegreeVertices)
  for edge in matching:
    mst.pushEdge(edge[0], edge[1], edge[2])
  tour = eulerianTour(mst)
  tour.append(0)
  return tour
  

In [217]:
def getTourWeight(graph, tour):
  weight = 0
  for i in range(len(tour) - 1):
    weight += graph.adjM[tour[i], tour[i + 1]]
  return weight

In [218]:
# Class 13 slide problem
graph = Graph(5)
graph.pushEdge(0, 1, 4)
graph.pushEdge(0, 2, 8)
graph.pushEdge(0, 3, 9)
graph.pushEdge(0, 4, 12)
graph.pushEdge(1, 2, 6)
graph.pushEdge(1, 3, 8)
graph.pushEdge(1, 4, 9)
graph.pushEdge(2, 3, 10)
graph.pushEdge(2, 4, 11)
graph.pushEdge(3, 4, 7)

approxTSPTour = approxTSP(graph)
print('Approximate TSP Algorithm')
print('Path:', approxTSPTour)
print('Total weight:', getTourWeight(graph, approxTSPTour))

print('\n')

christofidesTour = christofidesAlgorithm(graph)
print('Christofides Algorithm')
print('Path:', christofidesTour)
print('Total weight:', getTourWeight(graph, christofidesTour))

Approximate TSP Algorithm
Path: [0, 1, 3, 4, 2, 0]
Total weight: 38


Christofides Algorithm
Path: [0, 1, 2, 4, 3, 0]
Total weight: 37


In [219]:
@jit(nopython=True)
def euclidean_distance(point1, point2):
  return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

In [220]:
def read_tsp_file_input(file_path):
  points = []
  with open(file_path, 'r') as file:
    lines = file.readlines()
    read_coordinates = False
    for line in lines:
      if line.startswith('NODE_COORD_SECTION'):
        read_coordinates = True
        continue
      if read_coordinates and line.strip() and not line.startswith('EOF'):
        parts = line.strip().split()
        x, y = map(float, parts[1:])
        # np.append(points, (x, y))
        points.append((x, y))
  return points

In [221]:
def read_tsp_file_output(file_path):
  tour = []
  with open(file_path, 'r') as file:
    lines = file.readlines()
    read_tour = False
    for line in lines:
      if line.startswith('TOUR_SECTION'):
        read_tour = True
        continue
      if read_tour and line.strip() and not line.startswith('-1') and not line.startswith('EOF'):
        tour.append(int(line.strip()))
  return tour

In [222]:
@jit(nopython=True)
def add_edges(points, adjM):
  num_points = len(points)
  for point1_id in range(num_points):
    for point2_id in range(num_points):
      if point1_id != point2_id:
        distance = euclidean_distance(points[point1_id], points[point2_id])
        source = int(point1_id) - 1
        destination = int(point2_id) - 1
        adjM[source, destination] = distance
        adjM[destination, source] = distance

In [223]:
# Berlin 52
points = read_tsp_file_input('data/berlin52/berlin52.tsp')
graph = Graph(len(points))
add_edges(points, graph.adjM)

approxTSPTour = approxTSP(graph)
print('Approximate TSP Algorithm')
print('Path:', approxTSPTour)
print('Total weight:', getTourWeight(graph, approxTSPTour))

print('\n')

christofidesTour = christofidesAlgorithm(graph)
print('Christofides Algorithm')
print('Path:', christofidesTour)
print('Total weight:', getTourWeight(graph, christofidesTour))

print('\n')

optimalTour = read_tsp_file_output('data/berlin52/berlin52.opt.tsp')
optimalTour = [vertex - 1 for vertex in optimalTour]
print('Optimal Tour (Given)')
print('Path:', optimalTour)
print('Total weight:', getTourWeight(graph, optimalTour))

Approximate TSP Algorithm
Path: [0, 5, 40, 19, 29, 20, 51, 47, 34, 37, 38, 36, 22, 46, 44, 3, 13, 41, 31, 4, 2, 23, 10, 49, 9, 26, 25, 24, 45, 11, 50, 12, 35, 33, 32, 42, 14, 48, 27, 18, 21, 28, 30, 43, 17, 39, 6, 8, 7, 16, 1, 15, 0]
Total weight: 9775


Christofides Algorithm
Path: [0, 5, 40, 19, 29, 15, 1, 16, 20, 51, 47, 30, 43, 17, 39, 6, 8, 7, 31, 41, 13, 3, 4, 2, 23, 10, 9, 49, 26, 25, 11, 12, 50, 24, 45, 44, 46, 22, 36, 38, 35, 37, 34, 33, 32, 42, 14, 48, 18, 21, 28, 27, 0]
Total weight: 8801


Optimal Tour (Given)
Path: [0, 48, 31, 44, 18, 40, 7, 8, 9, 42, 32, 50, 10, 51, 13, 12, 46, 25, 26, 27, 11, 24, 3, 5, 14, 4, 23, 47, 37, 36, 39, 38, 35, 34, 33, 43, 45, 15, 28, 49, 19, 22, 29, 1, 6, 41, 20, 16, 2, 17, 30, 21]
Total weight: 26379


/usr/local/lib/python3.10/dist-packages/numba/core/ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'points' of function 'add_edges'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../../../tmp/ipykernel_1351445/1199228745.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [224]:
# pcb442
points = read_tsp_file_input('data/pcb442/pcb442.tsp')
graph = Graph(len(points))
add_edges(points, graph.adjM)

approxTSPTour = approxTSP(graph)
print('Approximate TSP Algorithm')
print('Path:', approxTSPTour)
print('Total weight:', getTourWeight(graph, approxTSPTour))

print('\n')

christofidesTour = christofidesAlgorithm(graph)
print('Christofides Algorithm')
print('Path:', christofidesTour)
print('Total weight:', getTourWeight(graph, christofidesTour))

print('\n')

optimalTour = read_tsp_file_output('data/pcb442/pcb442.opt.tsp')
optimalTour = [vertex - 1 for vertex in optimalTour]
print('Optimal Tour (Given)')
print('Path:', optimalTour)
print('Total weight:', getTourWeight(graph, optimalTour))

Approximate TSP Algorithm
Path: [0, 441, 440, 33, 65, 32, 64, 100, 439, 384, 125, 135, 148, 160, 171, 184, 172, 394, 397, 185, 173, 149, 150, 390, 136, 114, 113, 385, 387, 102, 101, 112, 124, 134, 147, 159, 170, 183, 398, 403, 225, 232, 236, 264, 267, 271, 274, 277, 279, 280, 426, 340, 339, 268, 269, 272, 275, 424, 438, 278, 281, 307, 306, 282, 308, 337, 431, 347, 348, 349, 350, 341, 351, 352, 353, 432, 354, 355, 356, 433, 357, 358, 359, 342, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 343, 346, 345, 344, 283, 309, 284, 310, 338, 285, 311, 286, 422, 419, 423, 312, 287, 313, 288, 314, 289, 315, 290, 316, 291, 317, 292, 318, 293, 319, 294, 320, 295, 321, 428, 296, 322, 427, 297, 298, 323, 299, 324, 300, 325, 429, 301, 326, 302, 327, 303, 328, 304, 329, 330, 331, 430, 332, 333, 334, 425, 335, 336, 305, 276, 265, 238, 233, 226, 405, 404, 399, 237, 1, 34, 66, 2, 35, 67, 3, 36, 68, 4, 37, 69, 5, 38, 70, 6, 39, 71, 7, 40, 72, 8, 41, 73, 9, 42, 74, 10, 43, 75, 97, 11,

In [225]:
# pcb442
points = read_tsp_file_input('data/brd14051/brd14051.tsp')
graph = Graph(len(points))
add_edges(points, graph.adjM)

approxTSPTour = approxTSP(graph)
print('Approximate TSP Algorithm')
print('Path:', approxTSPTour)
print('Total weight:', getTourWeight(graph, approxTSPTour))

print('\n')

christofidesTour = christofidesAlgorithm(graph)
print('Christofides Algorithm')
print('Path:', christofidesTour)
print('Total weight:', getTourWeight(graph, christofidesTour))

Approximate TSP Algorithm
Path: [0, 1, 23, 32, 48, 49, 81, 120, 118, 78, 109, 139, 162, 215, 193, 214, 259, 321, 351, 417, 466, 534, 560, 498, 538, 567, 599, 672, 726, 768, 832, 831, 820, 899, 961, 977, 1019, 1005, 955, 965, 1056, 1130, 1161, 1169, 1166, 1112, 1089, 1031, 991, 983, 934, 885, 819, 796, 803, 705, 666, 620, 552, 588, 591, 642, 667, 551, 550, 582, 565, 611, 645, 719, 730, 802, 842, 830, 863, 756, 700, 660, 619, 542, 607, 632, 459, 530, 524, 401, 372, 444, 302, 287, 344, 291, 262, 266, 271, 233, 206, 169, 175, 135, 129, 107, 127, 85, 86, 148, 76, 89, 57, 60, 94, 136, 112, 155, 184, 25, 644, 655, 519, 504, 414, 900, 860, 782, 770, 875, 743, 739, 702, 661, 598, 583, 646, 566, 500, 445, 431, 439, 493, 470, 381, 356, 432, 332, 299, 280, 268, 308, 721, 706, 731, 740, 789, 525, 584, 568, 605, 600, 641, 647, 561, 471, 450, 379, 370, 409, 340, 362, 354, 421, 316, 355, 357, 405, 418, 437, 465, 506, 476, 486, 412, 376, 406, 440, 463, 510, 563, 630, 656, 593, 460, 540, 553, 461, 447, 

KeyboardInterrupt: 